# Advent of Code - 2020 - Day 16


In [1]:
# Read data as one large string
data = ''
with open("inputs_day_16.txt", "r") as f:
  data = ''.join(f.readlines())

In [2]:
# Parse
data_groups = data.split('\n\n')

# Capture field rules in a dictionary
field_rules_raw = data_groups[0].split('\n')
field_rules = {}
for rule in field_rules_raw:
  rule_name = rule[:rule.find(':')]
  ranges = rule[rule.find(':') + 2:].split(' ')
  range_1 = [int(val) for val in ranges[0].split('-')]
  range_2 = [int(val) for val in ranges[2].split('-')]
  field_rules[rule_name] = [range_1, range_2]
  
field_rules

{'arrival location': [[35, 336], [358, 960]],
 'arrival platform': [[25, 632], [639, 970]],
 'arrival station': [[47, 442], [449, 955]],
 'arrival track': [[34, 461], [472, 967]],
 'class': [[41, 211], [217, 959]],
 'departure date': [[48, 572], [588, 965]],
 'departure location': [[44, 401], [415, 965]],
 'departure platform': [[29, 477], [484, 963]],
 'departure station': [[44, 221], [243, 953]],
 'departure time': [[48, 702], [719, 955]],
 'departure track': [[43, 110], [126, 951]],
 'duration': [[29, 500], [519, 969]],
 'price': [[39, 423], [440, 969]],
 'route': [[50, 264], [282, 958]],
 'row': [[50, 907], [920, 972]],
 'seat': [[27, 294], [315, 954]],
 'train': [[29, 813], [827, 962]],
 'type': [[45, 531], [546, 956]],
 'wagon': [[29, 283], [292, 957]],
 'zone': [[45, 518], [525, 974]]}

In [3]:
# Capture my ticket in a list
my_ticket_raw = data_groups[1]
my_ticket = [int(val) for val in my_ticket_raw[my_ticket_raw.find('\n') + 1:].split(',')]
print(my_ticket)

[89, 139, 79, 151, 97, 67, 71, 53, 59, 149, 127, 131, 103, 109, 137, 73, 101, 83, 61, 107]


In [4]:
# Capture nearby tickets as a list of lists
nearby_tickets_raw = data_groups[2]
nearby_tickets = nearby_tickets_raw[nearby_tickets_raw.find('\n') + 1:].split('\n')
nearby_tickets = [[int(val) for val in ticket.split(',')] for ticket in nearby_tickets]

print(nearby_tickets[:2])

[[749, 494, 864, 530, 921, 599, 370, 550, 323, 202, 821, 99, 783, 496, 90, 828, 65, 605, 725, 745], [729, 731, 400, 774, 600, 84, 645, 661, 730, 486, 582, 870, 207, 640, 844, 567, 326, 592, 390, 664]]


## Part 1

In [5]:
# Applies rule (list of two ranges)
# A range is a list of two numbers
# To a number
# Return true if passed
def apply_rule_to_number(number, rule):
  return ((number in range(rule[0][0], rule[0][1] + 1)) or (number in range(rule[1][0], rule[1][1] + 1)))


In [6]:
# Apply a set of rules to a ticket
# Return all numbers that make that ticket invalid 
# (in practice, there is only one number wrong with invalid tickets)

def get_ticket_errors(ticket, rules):

  # For each number in ticket, it keeps track of how many rules it failed
  rule_failures_counts = [0] * len(ticket)

  for rule in rules:
    for i, ticket_number in enumerate(ticket):
      rule_failures_counts[i] += 1 - int(apply_rule_to_number(ticket_number, rule))

  errors = []
  for i, rule_failures_count in enumerate(rule_failures_counts):
    if(rule_failures_count == len(rules)): # failed all rules
      errors.append(ticket[i])

  return errors


In [7]:
invalid_tickets_indices = []
probelmatic_values_total = []
for i, nearby_ticket in enumerate(nearby_tickets):
  probelmatic_values = get_ticket_errors(nearby_ticket, list(field_rules.values()))
  for probelmatic_value in probelmatic_values:
    probelmatic_values_total.append(probelmatic_value)
  if(len(probelmatic_values) > 0):
    invalid_tickets_indices.append(i)


ticket_scanning_error_rate = sum(probelmatic_values_total)
print('Number of Invalid tickets indexes:', len(invalid_tickets_indices))
print('Ticket scanning errors:', probelmatic_values_total)
print('Ticket scanning error rate:', ticket_scanning_error_rate)


Number of Invalid tickets indexes: 51
Ticket scanning errors: [986, 999, 977, 5, 982, 979, 985, 983, 992, 995, 994, 2, 987, 978, 8, 982, 986, 5, 19, 976, 12, 12, 16, 17, 4, 985, 18, 990, 13, 975, 23, 20, 22, 13, 996, 18, 17, 13, 986, 986, 13, 4, 992, 996, 992, 8, 983, 978, 987, 5, 997]
Ticket scanning error rate: 27911


## Part 2

I will not use Pandas for the logic, as I am hoping to do the actual logic with native Python and the Python Standard Library. However, Pandas allows for neat visualization of the ticket columns.

In [8]:
import pandas as pd  
valid_tickets = [nearby_tickets[i] for i, nearby_ticket in enumerate(nearby_tickets) if i not in invalid_tickets_indices]
df = pd.DataFrame(valid_tickets)  
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,749,494,864,530,921,599,370,550,323,202,821,99,783,496,90,828,65,605,725,745
1,729,731,400,774,600,84,645,661,730,486,582,870,207,640,844,567,326,592,390,664
2,658,564,639,901,754,589,352,373,102,677,54,949,596,316,93,648,109,676,499,416
3,203,527,234,99,933,695,771,142,864,702,875,758,876,359,741,605,668,52,376,886
4,872,457,109,217,162,786,805,788,168,684,937,672,828,632,943,108,336,769,804,911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,379,877,218,361,809,904,745,804,109,840,283,149,792,366,83,60,443,210,843,62
186,186,383,399,130,937,188,180,162,527,942,252,321,146,631,565,869,288,611,359,700
187,927,549,186,889,172,928,691,568,321,657,890,294,610,644,679,532,263,874,726,132
188,548,722,526,649,376,949,459,305,187,834,668,372,398,696,610,93,476,473,777,756


In [9]:
# Each item is a list of rules that are not broken by the elements of the corresponding column
rules_adhered_to_by_columns = {i : [] for i in range(df.shape[1])}

for i in range(len(valid_tickets[0])): # for number of columns
  column = [valid_tickets[j][i] for j in range(len(valid_tickets))] # grab the column
  for rule_key in field_rules.keys():
    rule_applies_counter = 0
    for column_element in column:
      if(apply_rule_to_number(column_element, field_rules[rule_key])):
        rule_applies_counter += 1

    if(rule_applies_counter == len(column)):
      rules_adhered_to_by_columns[i].append(rule_key)

for key in rules_adhered_to_by_columns:
    print(key, rules_adhered_to_by_columns[key])

0 ['departure location', 'departure station', 'departure platform', 'departure track', 'departure date', 'departure time', 'arrival station', 'arrival platform', 'arrival track', 'class', 'duration', 'seat', 'wagon', 'zone']
1 ['departure location', 'departure station', 'departure platform', 'departure track', 'departure date', 'departure time', 'arrival location', 'arrival station', 'arrival platform', 'arrival track', 'class', 'duration', 'price', 'route', 'row', 'seat', 'train', 'type', 'wagon', 'zone']
2 ['arrival platform', 'arrival track', 'class', 'duration', 'seat', 'wagon', 'zone']
3 ['departure location', 'departure station', 'departure platform', 'departure track', 'departure date', 'departure time', 'arrival location', 'arrival station', 'arrival platform', 'arrival track', 'class', 'duration', 'route', 'seat', 'train', 'type', 'wagon', 'zone']
4 ['departure location', 'departure station', 'departure platform', 'departure track', 'departure date', 'departure time', 'arrival

As we can see, each column may have multiple rules it complies with. We need to process this such that each coumn is asscoiated with a single rule. We can do this by starting with columns that only comply with a single rule. This means that it cannot apply to any other column, and so we remove its association with other column. This should lead to additional columns that will only have a single rule. We can repeat this process until all columns are associated with a single rule. This is not very efficient, but it will do for this problem.

In [10]:
already_removed = []

while(True):

  for i in rules_adhered_to_by_columns.keys():
    if(len(rules_adhered_to_by_columns[i]) == 1):
      rule = rules_adhered_to_by_columns[i][0]
      if(rule not in already_removed):
        already_removed.append(rule)
        for j in rules_adhered_to_by_columns.keys():
          if(len(rules_adhered_to_by_columns[j]) >= 2):
            if(rule in rules_adhered_to_by_columns[j]):
              rules_adhered_to_by_columns[j].remove(rule)

  counter = 0
  for key in rules_adhered_to_by_columns.keys():
    if(len(rules_adhered_to_by_columns[key]) == 1):
      counter += 1

  if(counter == len(field_rules)):
    break

for key in rules_adhered_to_by_columns:
    print(key, rules_adhered_to_by_columns[key])

0 ['arrival station']
1 ['row']
2 ['wagon']
3 ['arrival location']
4 ['type']
5 ['route']
6 ['departure date']
7 ['zone']
8 ['arrival track']
9 ['seat']
10 ['departure location']
11 ['departure time']
12 ['departure platform']
13 ['arrival platform']
14 ['price']
15 ['departure track']
16 ['duration']
17 ['departure station']
18 ['class']
19 ['train']


In [11]:
indexes = []
for i, key in enumerate(rules_adhered_to_by_columns.keys()):
  if(rules_adhered_to_by_columns[key][0][:9] == 'departure'):
    indexes.append(i)
    
prod = 1
for i, n in enumerate(my_ticket):
  if(i in indexes):
    prod *= n
prod

737176602479